![This is an image](https://www.diamondaircraft.com/fileadmin/_processed_/7/2/csm_DA40-NG_Header_Bild-_1__dcaa735d66.jpg)

In [13]:
import pandas as pd
import numpy as np
import random

Создадим искусственный набор данных о подержанных самолетах. В нашем датасете будет представлена информация о характеристиках и цене 1000 подержанных самолетов [четырех самых популярных типов ВС в АОН США в 2020 году](http://www.fi-aeroweb.com/General-Aviation.html):
- Cessna CE-172 Skyhawk SP (id 0)
- Cirrus SR22T (id 1)
- Piper Aircraft's PA-28-181 Archer III (id 2)
- Diamond Aircraft's DA40 (id 3)
***
К характеристикам самолета будут относиться:
1. **Возраст** (случайный)
2. **Крейсерская скорость (kmph)** (от производителя)
3. **Дальность полета (km)** (от производителя)
4. **Тип двигателя** (случайный):
    - бензиновый (id 0)
    - дизельный (id 1)
5. **Наличие дополнительных топливных баков** (случайный)
6. **Тип авионики** (случайный):
    - аналоговый (id 0)
    - цифровой (id 1)
***

In [21]:
n_samples = 1000

In [219]:
aircraft = np.random.choice(4, n_samples, p=[0.33, 0.26, 0.21, 0.2])
age = np.random.choice(15, n_samples)
engine_id = np.random.choice(2, n_samples)
aux_tanks = np.random.choice(2, n_samples, p=[0.7,0.3])
avionic_id = np.random.choice(2, n_samples)

# Крейсерская скорость определяется производителем
cruise_speed = []
for a in aircraft:
    if a==0:
        cruise_speed.append(226)
    if a==1:
        cruise_speed.append(339)
    if a==2:
        cruise_speed.append(231)
    if a==3:
        cruise_speed.append(272)
cruise_speed = np.array(cruise_speed)

# Дальность полета определяется производителем
range_km = []
for a in aircraft:
    if a==0:
        range_km.append(1185)
    if a==1:
        range_km.append(1943)
    if a==2:
        range_km.append(1241)
    if a==3:
        range_km.append(1741)
range_km = np.array(range_km)

# Цена (в USD) будет определяться ее значением, опредленным производителем, 
# с учетом возраста, с введенной случайной погрешностью
price = []
for a, y in zip(aircraft, age):
    if a==0:
        price.append(438000 - np.random.randint(15000,25000)*y^2)
    if a==1:
        price.append(779000 - np.random.randint(15000,25000)*y^2)
    if a==2:
        price.append(502000 - np.random.randint(15000,25000)*y^2 )
    if a==3:
        price.append(495000 - np.random.randint(15000,25000)*y^2)
price = np.array(price)

In [223]:
data = pd.DataFrame({'aircraft_id': aircraft, 'age': age, 'cruise_speed': cruise_speed, 'range': range_km,
                    'engine_id': engine_id, 'aux_tanks': aux_tanks, 'avionic_id': avionic_id,
                    'price': price})
data.head()

,aircraft_id,age,cruise_speed,range,engine_id,aux_tanks,avionic_id,price
0,3,4,272,1741,1,1,1,415834
1,0,12,226,1185,0,1,0,223622
2,3,1,272,1741,1,0,0,479600
3,3,3,272,1741,1,1,1,428017
4,1,13,339,1943,1,0,0,485796


# Моделирование

In [225]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [260]:
data.columns

Index(['aircraft_id', 'age', 'cruise_speed', 'range', 'engine_id', 'aux_tanks',
       'avionic_id', 'price'],
      dtype='object')

In [279]:
X = data[['aircraft_id', 'age', 'cruise_speed', 'range', 'engine_id', 'aux_tanks','avionic_id']]
y = data['price']

lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X,y)

print('WEIGHTS')
print([coef for coef in lin_reg.coef_])
print('BIAS')
print(lin_reg.intercept_)

pred = lin_reg.predict(X)
print('ERROR')
print(mean_squared_error(y,pred))

WEIGHTS
[34275.20772008293, -19926.72450726459, 7330.3499438152085, -687.004980607909, -674.5387126971483, -165.55102503499918, 2671.5386542959086]
BIAS
-407235.6057092707
ERROR
574082732.9069809


In [280]:
X = data[['aircraft_id', 'age', 'cruise_speed', 'range']]
y = data['price']

lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X,y)

print('WEIGHTS')
print([coef for coef in lin_reg.coef_])
print('BIAS')
print(lin_reg.intercept_)

pred = lin_reg.predict(X)
print('ERROR')
print(mean_squared_error(y,pred))

WEIGHTS
[34166.95398068364, -19946.70440674039, 7320.964526188809, -685.5925700169562]
BIAS
-405628.1201252922
ERROR
575999027.9021752


Создадим новый признак

In [281]:
data['mult'] = data.range * data.cruise_speed
data.head()

,aircraft_id,age,cruise_speed,range,engine_id,aux_tanks,avionic_id,price,mult
0,3,4,272,1741,1,1,1,415834,473552
1,0,12,226,1185,0,1,0,223622,267810
2,3,1,272,1741,1,0,0,479600,473552
3,3,3,272,1741,1,1,1,428017,473552
4,1,13,339,1943,1,0,0,485796,658677


In [282]:
X = data[['mult']]
y = data['price']

lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X,y)

print('WEIGHTS')
print([coef for coef in lin_reg.coef_])
print('BIAS')
print(lin_reg.intercept_)

pred = lin_reg.predict(X)
print('ERROR')
print(mean_squared_error(y,pred))

WEIGHTS
[0.7695094050399032]
BIAS
91520.60133420693
ERROR
11620801749.841412
